In [ ]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("cq01/mawps-asdiv-a_svamp")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install --upgrade pip

!pip uninstall unsloth -y
!pip uninstall torch torchvision -y
!pip uninstall xformers -y

!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"
!pip install torch==2.4.1 torchvision==0.19.1
!pip install --upgrade xformers==0.0.28.post1

  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.1.9
    Uninstalling nvidia-cusolver-cu12-11.6.1.9:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.1.9
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0
    Uninstalling torch-2.5.0:
      Successfully uninstalled torch-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.0+cu121 requires torch==2.5.0, but you have torch 2.4.1 which is incompatible.
xformers 0.0.28.post2 requires torch==2.5.0, but you have torch 2.4.1 which is incompatible.
  Using cached xformers-0.0.28.post1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
Using cached xformers-0.0.28.post1-cp310-cp310-manylinux_2_28_x86_64.whl (16.7 MB)
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.28.post2
    Unins

In [ ]:
!python -m xformers.info
# !nvidia-smi

xFormers 0.0.28.post1
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF-pt:            available
memory_efficient_attention.cutlassB-pt:            available
memory_efficient_attention.fa2F@v2.5.6-pt:         available
memory_efficient_attention.fa2B@v2.5.6-pt:         available
memory_efficient_attention.fa3F@0.0.0:             unavailable
memory_efficient_attention.fa3B@0.0.0:             unavailable
memory_efficient_attention.triton_splitKF:         unavailable
indexing.scaled_index_addF:                        unavailable
indexing.scaled_index_addB:                        unavailable
indexing.index_select:                             unavailable
sequence_parallel_fused.write_values:              available
sequence_parallel_fused.wait_values:       

In [ ]:
import os
import torch
import numpy as np
import pandas as pd

In [ ]:
for key, value in ds.items():
    print(f"Length of '{key}': {len(value)}")

Length of 'train': 3138
Length of 'validation': 1000


In [ ]:
def generate_full_question(example):
    question = example["Question"]
    numbers = example["Numbers"]
    for i, num in enumerate(numbers):
        placeholder = f"number{i}"
        # Check if the number is an integer
        if num.is_integer():
            num_str = str(int(num))
        else:
            num_str = str(num)
        question = question.replace(placeholder, num_str)
    example["Full Question"] = question
    return example

# Apply the transformation to the train and validation sets
ds = ds.map(generate_full_question)

# Now you can access the transformed dataset with the new "Full Question" column
print(ds["train"][23]["Full Question"])

A construction company ordered 0.16666666666666666 ton of concrete , 0.16666666666666666 ton of bricks , and 0.5 ton of stone . How many tons of material did the company order in all ?


In [ ]:
# !pip install peft trl
# !pip install bitsandbytes
# !pip uninstall torch torchvision -y
# !pip install torch==2.4.1 torchvision==0.19.1
# !pip install --upgrade accelerate einops
import torchvision
import torch
# !pip3 install -q --upgrade accelerate einops xformers

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

In [ ]:
# !pip install unsloth
# !pip install ninja
# !pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu118
# !python -m xformers.info
# !pip3 uninstall xformers -y
# !pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu124
# !python -m xformers.info

!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2024.11.5
Uninstalling unsloth-2024.11.5:
  Successfully uninstalled unsloth-2024.11.5
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-x52t449t/unsloth_432786833cb5489d8477768cde98b938
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-x52t449t/unsloth_432786833cb5489d8477768cde98b938
  Resolved https://github.com/unslothai/unsloth.git to commit 0c8c5ed81e423658ab9ae81eac5aab8d18f5d7af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.5-py3-none-any.whl size=161005 sha256=869b5f38dbdff6e9c39b4ca8bb6353422c01cbf3387a7eaf3a55a8c0a33beb53
  Stored in directory: /tmp/pip-ephem-wheel-cache-z3l7b7o0/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

def load_model():
    """Load Gemma-2-9b model and tokenizer with LoRA applied."""
    max_seq_length = 2048
    load_in_4bit = True

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/gemma-2-9b",
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=load_in_4bit,
        device_map="cuda:0",
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=32,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None
    )

    return model, tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
# Load the base Gemma-2-9b model
model, tokenizer = load_model()

==((====))==  Unsloth 2024.11.5: Fast Gemma2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.5 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [ ]:
def format_answer(answer):
    """Convert answer to float format."""
    try:
        return f"the answer is {float(answer)}"
    except:
        # Fallback for any non-numeric answers
        return f"the answer is {answer}"

def format_mawps_data(examples):
    """Preprocess MAWPS-ASDIV-A dataset for training."""
    questions = examples["Full Question"]
    answers = examples["Answer"]
    texts = []

    prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

### Input:
{input}

### Response:
{response}"""

    for question, answer in zip(questions, answers):
        text = prompt_template.format(
            input=question,
            response=format_answer(answer)
        )
        texts.append(text)

    return {"text": texts}

# Process the dataset
formatted_dataset = ds.map(
    format_mawps_data,
    remove_columns=ds["train"].column_names,
    batched=True
)

In [ ]:
def verify_formatting(dataset, num_examples=2):
    """Verify the formatting of the processed dataset."""
    print("Sample formatted examples:")
    for i in range(min(num_examples, len(dataset))):
        print(f"\nExample {i+1}:")
        print(dataset[i]["text"])
        print("-" * 50)

print("Dataset format verification:")
verify_formatting(formatted_dataset["validation"])

Dataset format verification:
Sample formatted examples:

Example 1:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

### Input:
julia played tag with 18 kids on monday . she played tag with 10 kids on tuesday . how many more kids did she play with on monday than on tuesday ?

### Response:
the answer is 8.0
--------------------------------------------------

Example 2:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

### Input:
julia played tag with 11 kids on monday . she 

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

30

In [ ]:
training_config = TrainingArguments(
    output_dir="./math_model_output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    seed=3407,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["validation"],
    tokenizer=tokenizer,
    args=training_config,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=2,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
print("Starting training...")

trainer.train()

# Save the final model
trainer.save_model("./math_model_final")

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,138 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 294
 "-____-"     Number of trainable parameters = 54,018,048
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: swayamagrawal1004. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
0,0.448200,0.612162


E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6] failed while attempting to run meta for aten.add_.Tensor
E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6] Traceback (most recent call last):
E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6]   File "/usr/local/lib/python3.10/dist-packages/torch/_subclasses/fake_tensor.py", line 1757, in _dispatch_impl
E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6]     r = func(*args, **kwargs)
E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6]   File "/usr/local/lib/python3.10/dist-packages/torch/_ops.py", line 667, in __call__
E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6]     return self_._op(*args, **kwargs)
E1112 10:32:35.145000 139509862346752 torch/_subclasses/fake_tensor.py:1761] [1/6]   File "/usr/local/lib/python3.10/dist-packages/torch/_meta_r

TorchRuntimeError: Failed running call_function <built-in function iadd>(*(FakeTensor(..., device='cuda:0', size=(s10, s2*s3, s19, s11),
           dtype=torch.float16), FakeTensor(..., device='cuda:0', size=(s20, 1, s21, s22), dtype=torch.float16)), **{}):
Attempting to broadcast a dimension of length s22 at -1! Mismatching argument at index 1 had torch.Size([s20, 1, s21, s22]); but expected shape should be broadcastable to [s10, s2*s3, s19, s11]

from user code:
   File "/usr/local/lib/python3.10/dist-packages/unsloth/kernels/flex_attention.py", line 67, in slow_attention_softcapping
    A += causal_mask[:q_len, :q_len]

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


In [ ]:
model.save_pretrained("lmao/models/mathGemma-2-9b")
tokenizer.save_pretrained("lmao/models/mathGemma-2-9b")

In [ ]:
import shutil

shutil.make_archive("lmao", 'zip', "lmao")

In [ ]:
from google.colab import files

files.download("lmao.zip")

In [ ]:
!ls
!pwd